## How to use Simple Coder

### Basic Implementation
Let's start with a very simple implementation to demonstrate how you can generate an output file with some simple instructions.

We will create an ```agenda``` object to organize and store our instructions, then use that to run our ```SimpleCoder``` agent.

Each ```SimpleCoder``` instance takes one instruction item to run, and outputs one file. We will handle complex agendas, messaging, etc somewhere else. This will allow us to simplify our code and focus on refining the core functionality, which features recursive calling that will be much easier to understand and troubleshoot without the additional overhead.

## Create Virtual Environment

```bash
python -m venv venv
source venv/bin/activate
```

## Install Dependencies

```bash
make install
```

## Import

In [1]:
from simplecoder import SimpleCoder

# Directory where code will be saved and read from
C_WORKING_DIR = "temp/"

In [2]:
# We organize our instructions in a dict. 
# Later, we can easily load this from a JSON file or a database.
agenda = {
    "requirements": "I need a script that gets and prints the current weather in New York without requiring an API key.",
    "output_file_name": "weather.sh",
    "working_dir": C_WORKING_DIR
}

# Create a coder agent using the agenda
coder = SimpleCoder(**agenda)

# Run the agent
await coder.run()

Epoch: 0
Epoch: 1


'weather.sh'

The basic properties to our agenda are:
- ```requirements``` - State what you need or what you want the agent to create
- ```output_file_name``` - This is where we will write the code output. If the file already exists, the contents will be loaded as our initial source code for refactoring. Otherwise it will be created.
- ```working_dir``` - This is the base path for our input and output files

The agent will iterate at least 2 times, or until the LLM thinks it is complete and issues a stop code. This allows the LLM to analyze it's own output and make changes, increasing the quality of our output.

#### check the output.

In [ ]:
!cat {agenda["working_dir"] + agenda["output_file_name"]}

#### run the file

In [14]:
!. {agenda["working_dir"] + agenda["output_file_name"]}

Current weather in New York:
Temperature: +71°F
Condition: Overcast 


## Using references 
We have illustrated how we can create a new file or refactor an existing one. 

However, projects often require multiple files, some of which are interdependant. It is very cumbersome to add all this source code to a chat bot, with your refactoring instructions.

With SimpleCoder, we can do this by adding a new key to our agenda:

```"input_file_name_list": ["project_readme.md", "project_file.py"]```

This allows us to specify a list of file names to be read in as references for our agent.

Let's see how this works. We construct the following problem to illustrate the implementation of a class declared in another file.

In [ ]:
# Create an agenda list
# Note that the second element takes the output of the first element as input via "input_file_name_list"
agenda = [
    {
        "requirements": "I need a python class, WebLoader that downloads a URL, extracts text, and saves it to a file",
        "output_file_name": "web_loader.py",
        "working_dir": C_WORKING_DIR,
        "force_code": True
    },
    {
        "requirements": "I need a python class WebScraper that uses WebLoader to scrape google for news mentioning AI",
        "output_file_name": "web_scraper.py",
        # This is a list of files that will be loaded as input for the agent. The agent is instructed to use these for reference.
        "input_file_list": ["web_loader.py"],
        "working_dir": C_WORKING_DIR,
        "force_code": True
    }
]

# Iterate over our agenda and run an agent for each item
for i, item in enumerate(agenda):
    print(f"Agenda item #{i}: {item['output_file_name']} - starting")

    # Create and run the agent
    coder = SimpleCoder(**item)
    print( await coder.run() )
    
    print(f"Agenda item {i} - finished\n\n")

If you try the given example and check the output files, you should see that the ```WebScraper``` class will include an import and implementation of the ```WebLoader``` class.

In [ ]:
!cat {C_WORKING_DIR + agenda[0]["output_file_name"]}
!cat {C_WORKING_DIR + agenda[1]["output_file_name"]}

## Run the output code, and marvel!

In [ ]:
!python3 {C_WORKING_DIR + agenda[-1]["output_file_name"]}

Observe the marvelous runtime and/or dependency errors!

This is actually a challanging problem for the LLM because it does not know the format of the data it will be scraping. It can adopt several solutions for this, some of which employ BeautifulSoup or regex patterns. Some of these solutions are even functional!

### The goal of this excercise, however, is to demonstrate several things:
1. The agent can create several interdependant files
2. The agent uses proper class names and function singatures across all files
3. Web scraping is not easy, man

## Limitations 

- Produced code may require dependencies.

- Produced code may contain placeholder functions for mechanisms that are too complex, requiring several more iterations to complete the full output.

- Because the agent simply examines the code and does not actually run it to debug, it can generate some syntactical or logical errors.

- The agent may iterrate endlessly, thinking that the requirements are not properly implemented, and will keep making small adjustments until it reaches the max iteration limit.